In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
#clean reviews
def docs_to_word_list(series):
  return series.map(lambda x: re.findall('\w+', x.lower()))

# Document Similarity Using Word Embedding

In [ ]:
attributes = ['citrus', 'tropical', 'hops']
reviews = pd.read_csv('beer_reviews.csv')[['name', 'review']]

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load("en_core_web_md")
import en_core_web_md
nlp = en_core_web_md.load()
# remove generic beer words
stopwords = set(['beer', 'brew', 'ale', 'mead'])
reviews.loc[:, 'split_review'] = reviews.review.map(lambda x: re.findall('\w+', x.lower()))
reviews.loc[:, 'split_review'] = reviews.split_review.map(lambda x: [word for word in x if word not in stopwords])
reviews.loc[:, 'review'] = reviews.split_review.map(lambda x: ' '.join(x))

attr_vector = nlp(' '.join(attributes))
reviews['embedding_score'] = reviews.review.map(lambda x: nlp(x).similarity(attr_vector))
reviews.head()

<ipython-input-6-7687fdac9cc1>:11: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  reviews['embedding_score'] = reviews.review.map(lambda x: nlp(x).similarity(attr_vector))


,name,review,split_review,embedding_score
0,Kentucky Brunch Brand Stout,so ok on the real i gave up a ton for a couple...,"[so, ok, on, the, real, i, gave, up, a, ton, f...",0.325409
1,Kentucky Brunch Brand Stout,poured into a snifter glass a dark chocolaty c...,"[poured, into, a, snifter, glass, a, dark, cho...",0.542600
2,Kentucky Brunch Brand Stout,if there s any that deserves the rating of a 5...,"[if, there, s, any, that, deserves, the, ratin...",0.346571
3,Kentucky Brunch Brand Stout,this has been on my bucket list for a while th...,"[this, has, been, on, my, bucket, list, for, a...",0.335161
4,Kentucky Brunch Brand Stout,i didnt think i was going to give it a perfect...,"[i, didnt, think, i, was, going, to, give, it,...",0.418328


In [ ]:
reviews.sort_values(by='embedding_score', ascending=False, inplace=True)
reviews.head()

,name,review,split_review,embedding_score
2304,Triple Sunshine,aroma big and sweet tropical fruit smell pinea...,"[aroma, big, and, sweet, tropical, fruit, smel...",0.743084
614,Double Barrel V.S.O.J.,boozy concentrated sweet drippy raisins figs d...,"[boozy, concentrated, sweet, drippy, raisins, ...",0.740573
568,Clover,golden yellow pour miniscule head white wine o...,"[golden, yellow, pour, miniscule, head, white,...",0.713571
5467,Cellarman Barrel Aged Saison,the aroma offers apricot tart stone fruit oak ...,"[the, aroma, offers, apricot, tart, stone, fru...",0.701120
3392,Dorothy (Wine Barrel Aged),gold pour hazy larger head white wine grape oa...,"[gold, pour, hazy, larger, head, white, wine, ...",0.693241


# Sentiment Analysis Using Vader

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
vader = SentimentIntensityAnalyzer()
#adding context
newWords = {'bitter': 0.75, 'hoppy': 1, 'opaque': 1, 'sour': 1.5, 'boozy': -2, 'alcohol': -2}
vader.lexicon.update(newWords)

reviews.loc[:, 'sentiment'] = reviews.review.map(lambda x: vader.polarity_scores(x)['compound'])
reviews.head()

,name,review,split_review,embedding_score,sentiment
2304,Triple Sunshine,aroma big and sweet tropical fruit smell pinea...,"[aroma, big, and, sweet, tropical, fruit, smel...",0.743084,0.5245
614,Double Barrel V.S.O.J.,boozy concentrated sweet drippy raisins figs d...,"[boozy, concentrated, sweet, drippy, raisins, ...",0.740573,0.5574
568,Clover,golden yellow pour miniscule head white wine o...,"[golden, yellow, pour, miniscule, head, white,...",0.713571,0.8910
5467,Cellarman Barrel Aged Saison,the aroma offers apricot tart stone fruit oak ...,"[the, aroma, offers, apricot, tart, stone, fru...",0.701120,0.0000
3392,Dorothy (Wine Barrel Aged),gold pour hazy larger head white wine grape oa...,"[gold, pour, hazy, larger, head, white, wine, ...",0.693241,0.0000


In [ ]:
# evenly weighed recommendation score
reviews['recommendation_score'] = reviews.embedding_score * reviews.sentiment
output = reviews[['name', 'embedding_score', 'sentiment', 'recommendation_score']].groupby(by=['name']).mean()
output.sort_values(by='recommendation_score', ascending=False, inplace=True)
output.head()

,embedding_score,sentiment,recommendation_score
name,,,
Beyond Good And Evil,0.429560,0.769594,0.361542
Coconut Vibes - Barrel-Aged,0.475621,0.719117,0.350645
Framboise Du Fermier,0.420520,0.779404,0.337857
Double Galaxy,0.436009,0.728540,0.331849
Montmorency Vs Balaton,0.434994,0.710213,0.326149


# Our Recommendation
Based on your preferences for a citrus-y, tropical, hoppy beer, we have three brews for you to try out. All three should be somewhere in the ballpark given your exact tasting notes and are as follows:
1. "Beyond Good and Evil"
2. "Coconut Vibes - Barrel-Aged"
3. "Framboise Du Fermier"

Keep in mind that our current recommendation model is based on similarity between your sought after attributes and others' reviews, so it will yield robust recommendations that will be somewhere in the range of your interests, but may not be an exact match.

In [ ]:
from collections import defaultdict
# represent reviews as a vector of 3 chosen attributes
def word2vec(words, attributes):
  word_counts = defaultdict(int)
  for word in words:
    if word in attributes:
      word_counts[word] += 1
  result = np.array([word_counts[key] for key in attributes]).reshape(1, -1)
  return result

In [ ]:
reviews.loc[:, 'attr_vector'] = reviews.split_review.map(lambda x: word2vec(x, attributes))


##### Checking top 3 recommendations for keyword attributes...

In [ ]:
reviews[reviews.name == 'Beyond Good And Evil']

,name,review,split_review,embedding_score,sentiment,recommendation_score,attr_vector
2753,Beyond Good And Evil,smells like hot buttered rum and coffee liquor...,"[smells, like, hot, buttered, rum, and, coffee...",0.647103,0.9639,0.623742,"[[0, 0, 0]]"
2759,Beyond Good And Evil,2014 batch enjoyed in 2017 and it was still am...,"[2014, batch, enjoyed, in, 2017, and, it, was,...",0.612021,0.9797,0.599597,"[[0, 0, 0]]"
2749,Beyond Good And Evil,fantastic dark brown to black appearance aroma...,"[fantastic, dark, brown, to, black, appearance...",0.558311,0.9717,0.542510,"[[0, 0, 0]]"
2760,Beyond Good And Evil,wow just wow big chewy chocolate covered raisi...,"[wow, just, wow, big, chewy, chocolate, covere...",0.533682,0.9657,0.515377,"[[0, 0, 0]]"
2748,Beyond Good And Evil,quite possibly the best stout i have ever had ...,"[quite, possibly, the, best, stout, i, have, e...",0.528424,0.9765,0.516006,"[[0, 0, 0]]"
2754,Beyond Good And Evil,2018 version poured from the bottle into a sni...,"[2018, version, poured, from, the, bottle, int...",0.515935,0.8860,0.457118,"[[0, 0, 0]]"
2752,Beyond Good And Evil,maple explosion 20 months in barrel makes it s...,"[maple, explosion, 20, months, in, barrel, mak...",0.510729,0.5994,0.306131,"[[0, 0, 0]]"
2758,Beyond Good And Evil,drank 5 1 15 loads of sweet thick maple syrup ...,"[drank, 5, 1, 15, loads, of, sweet, thick, map...",0.501211,0.9551,0.478707,"[[0, 0, 0]]"
2764,Beyond Good And Evil,how is this not higher on the list this is lit...,"[how, is, this, not, higher, on, the, list, th...",0.463713,0.8402,0.389612,"[[0, 0, 0]]"
2755,Beyond Good And Evil,quite possibly the perfect ba stout in my mind...,"[quite, possibly, the, perfect, ba, stout, in,...",0.432684,0.8993,0.389113,"[[0, 0, 0]]"


In [ ]:
reviews[reviews.name == 'Coconut Vibes - Barrel-Aged']

,name,review,split_review,embedding_score,sentiment,recommendation_score,attr_vector
352,Coconut Vibes - Barrel-Aged,pours pitch black wafts of sweet coconut choco...,"[pours, pitch, black, wafts, of, sweet, coconu...",0.627514,0.8655,0.543113,"[[0, 0, 0]]"
348,Coconut Vibes - Barrel-Aged,pours a deep dark brown basically opaque but a...,"[pours, a, deep, dark, brown, basically, opaqu...",0.541340,0.8641,0.467771,"[[0, 0, 0]]"
355,Coconut Vibes - Barrel-Aged,750ml white wax bottle served into a snifter g...,"[750ml, white, wax, bottle, served, into, a, s...",0.519573,0.9832,0.510844,"[[0, 0, 0]]"
349,Coconut Vibes - Barrel-Aged,thick gooey delicious just tastes magnificent ...,"[thick, gooey, delicious, just, tastes, magnif...",0.502010,0.9765,0.490213,"[[0, 0, 0]]"
356,Coconut Vibes - Barrel-Aged,i never thought i d get to try this it was pre...,"[i, never, thought, i, d, get, to, try, this, ...",0.468160,0.6124,0.286701,"[[0, 0, 0]]"
358,Coconut Vibes - Barrel-Aged,pours thick and viscous upfront chocolate coco...,"[pours, thick, and, viscous, upfront, chocolat...",0.457144,0.8984,0.410698,"[[0, 0, 0]]"
351,Coconut Vibes - Barrel-Aged,super thick awesome coconut the whole way and ...,"[super, thick, awesome, coconut, the, whole, w...",0.449195,0.9477,0.425702,"[[0, 0, 0]]"
353,Coconut Vibes - Barrel-Aged,pours thick black with little coconut floaties...,"[pours, thick, black, with, little, coconut, f...",0.448796,0.9861,0.442558,"[[0, 0, 0]]"
357,Coconut Vibes - Barrel-Aged,first review and stout in a minute in the proc...,"[first, review, and, stout, in, a, minute, in,...",0.428742,0.5627,0.241253,"[[0, 0, 0]]"
350,Coconut Vibes - Barrel-Aged,this is right up there with my perfect beers i...,"[this, is, right, up, there, with, my, perfect...",0.427229,0.5423,0.231686,"[[0, 0, 0]]"


In [ ]:

reviews[reviews.name == 'Framboise Du Fermier']

,name,review,split_review,embedding_score,sentiment,recommendation_score,attr_vector
946,Framboise Du Fermier,deep purplish red color very nice carb level a...,"[deep, purplish, red, color, very, nice, carb,...",0.546333,0.9398,0.513444,"[[0, 0, 0]]"
953,Framboise Du Fermier,deep red color and nice thick head well carbed...,"[deep, red, color, and, nice, thick, head, wel...",0.507825,0.9333,0.473953,"[[0, 0, 0]]"
950,Framboise Du Fermier,pouted from a 750 ml bottle the color is very ...,"[pouted, from, a, 750, ml, bottle, the, color,...",0.506388,0.9493,0.480714,"[[0, 0, 0]]"
956,Framboise Du Fermier,wow absolutely fantastic deep ruby red magenta...,"[wow, absolutely, fantastic, deep, ruby, red, ...",0.499747,0.9647,0.482106,"[[0, 0, 0]]"
947,Framboise Du Fermier,this was one of my top three from the 2015 hf ...,"[this, was, one, of, my, top, three, from, the...",0.496688,0.8957,0.444883,"[[0, 0, 0]]"
957,Framboise Du Fermier,the nose on this was absolutely incredible hug...,"[the, nose, on, this, was, absolutely, incredi...",0.490110,0.7674,0.376110,"[[0, 0, 0]]"
960,Framboise Du Fermier,part of a share in pittsburgh pours deep red w...,"[part, of, a, share, in, pittsburgh, pours, de...",0.476617,0.9795,0.466846,"[[0, 0, 0]]"
944,Framboise Du Fermier,the perfect don t drink too cold once it warms...,"[the, perfect, don, t, drink, too, cold, once,...",0.475761,0.9694,0.461202,"[[0, 0, 0]]"
959,Framboise Du Fermier,fresh picked raspberry on the nose fresh but m...,"[fresh, picked, raspberry, on, the, nose, fres...",0.472014,0.5187,0.244834,"[[0, 0, 0]]"
964,Framboise Du Fermier,l pours a bright ruby red that recedes into a ...,"[l, pours, a, bright, ruby, red, that, recedes...",0.462837,0.7096,0.328429,"[[0, 0, 0]]"


# Embeddings vs. Cosine Similarity

Looking at our output using SpaCy for embeddings, we were able to get some higher similarity scores to recommend for our customer. However, looking at the above counts of the actual attributes requested, we can see that NO reviews out of the top 3 beers had *any* mention of the attributes!

When trying to recommend products as specific as beer, it is probably better to use regular cosine similarity to look for exact matches. With word embeddings, words can be categorized as being "similar" due to appearing in similar contexts, rather than actually having similar characterstics. This phenomenon might be model-dependent, and modern LLMs may have fixed this issue, but using a library like SpaCy certainly had this limitation. For example, words like "coconut" have high similarity with "tropical," which normally would be great, but in the niche world of beer, those two lead to very different flavors.

Furthermore, before manually removing words such as "beer," we saw high document similarity with reviews that read as "amazing beer" because of beer's high similarity with a word like "hops."

Due to all these reasons, we believe cosine similarity is preferable for a beer review system, while word embeddings are much better in a more general sense (i.e. product recommendations on Amazon).

# Task G

In [ ]:
top_prods = reviews[['name', 'sentiment', 'embedding_score']].groupby(by=['name']).mean()
top_prods.sort_values(by='sentiment', ascending=False, inplace=True)
top_prods.head()

,sentiment,embedding_score
name,,
Double Sunshine,0.787810,0.408408
Everett,0.783767,0.352177
Framboise Du Fermier,0.779404,0.420520
Beyond Good And Evil,0.769594,0.429560
Cutting Tiles - Mosaic,0.748735,0.410941


In [ ]:
top_fits = reviews[['name', 'sentiment', 'embedding_score']].groupby(by=['name']).mean()
top_fits.sort_values(by='embedding_score', ascending=False, inplace=True)
top_fits.head()

,sentiment,embedding_score
name,,
Saison Du Fermier,0.604560,0.476699
Coconut Vibes - Barrel-Aged,0.719117,0.475621
Monster Tones,0.529267,0.465605
La Fosse,0.390630,0.456108
Speedway Stout - Vietnamese Coffee,0.700700,0.448298


By looking at the above tables, we can see there is no overlap between the top sentiments and top similarity scores. If we were to grab the 3 highest rated beers in terms of sentiment as our recommendation, we wouldn't have had the best results, but they also would not have been terrible. "Beyond Good and Evil," our top beer based on a combination of sentiment and similarity, still made the top 5 in terms of sentiment. Additionally, the top rated beer would not have been a *bad* choice, as it was only .04 away in similarity from the 5th best score in terms of similarity.

>Unrelated, but we once again see the shortcomings of word embeddings in the context of beer here - a stout is curiously the 5th best match for someone who enjoys hoppy, citrusy beers.

### Now looking at ratings...

In [ ]:
ratings = pd.read_csv('beer_reviews.csv')[['name', 'rating']]
ratings.head()

,name,rating
0,Kentucky Brunch Brand Stout,5.0
1,Kentucky Brunch Brand Stout,5.0
2,Kentucky Brunch Brand Stout,5.0
3,Kentucky Brunch Brand Stout,5.0
4,Kentucky Brunch Brand Stout,5.0


In [ ]:
grouped_beers = ratings.groupby('name').agg({'rating': ['count', 'mean']})
grouped_beers.sort_values([('rating','mean'),('rating','count')], ascending = False)

rating          
                                                    count      mean
name                                                               
Heady Topper                                           46  5.000000
Breakfast Stout                                        43  5.000000
Pliny The Elder                                        41  5.000000
KBS                                                    37  5.000000
Green                                                  36  5.000000
...                                                   ...       ...
Truth                                                  16  4.530625
Label Us Notorious - Calvados BA Big Poppa              7  4.520000
Leaner                                                 17  4.511765
Starry Noche                                           40  4.505500
Bourbon County Brand Two-Year Barleywine Reserv...     32  4.471250

[249 rows x 2 columns]

In [ ]:
top_beers = top_fits[(top_fits.index == 'Heady Topper') | (top_fits.index == 'Breakfast Stout') | (top_fits.index == 'Pliny The Elder')]
top_beers.sort_values(by='embedding_score', ascending=False, inplace=True)
top_beers.head()

<ipython-input-22-7e1bf6c08946>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_beers.sort_values(by='embedding_score', ascending=False, inplace=True)


,sentiment,embedding_score
name,,
Pliny The Elder,0.654861,0.383352
Heady Topper,0.596035,0.383337
Breakfast Stout,0.660802,0.365951


When we do the analysis on the top three beers, we can see that they have lower similarity scores for the actual attributes we're looking for. Due to this, they may not be the best match. None of these top beers are in the top embedding or sentiment scores. While we believe cosine is better than embedding, we believe embedding is still better than just offering the top 3 rated beers due to the difference in characteristics.

In summary, even though this may be the top 3 beers in ratings, they don't necessarily align with the attributes that the customer will be looking for.

# Task H

In [ ]:
grouped_beers.sort_values([('rating','count'),('rating','mean')], ascending = False, inplace=True)
grouped_beers

rating          
                                       count      mean
name                                                  
Kaggen! Stormaktsporter                   75  4.765333
Yellow Bus                                66  4.590758
KBS - Maple Mackinac Fudge                58  4.926897
BDCS                                      48  4.843333
CitraQuench'l                             48  4.792500
...                                      ...       ...
§ucaba                                     4  5.000000
Beatification                              3  5.000000
Ephraim                                    3  5.000000
Everett                                    3  5.000000
Fourth Dementia - Bourbon Barrel-Aged      2  5.000000

[249 rows x 2 columns]

In [ ]:
#from csv
top_attributes = ['dark', 'chocolate', 'bourbon', 'coffee']
top_attribute_freqs = {'dark': 789, 'chocolate': 771, 'bourbon': 636, 'coffee': 608}
chosen_beers = list(grouped_beers.index[0:10])

In [ ]:
# brand to attribute co-occurences
chosen_brands_reviews = reviews[reviews.name.isin(chosen_beers)]
chosen_brands_reviews.loc[:, 'attr_counts'] = chosen_brands_reviews.split_review.map(lambda x: word2vec(x, top_attributes))
chosen_brands_reviews.loc[:, 'dark'] = chosen_brands_reviews.attr_counts.map(lambda x: x[0][0])
chosen_brands_reviews.loc[:, 'chocolate'] = chosen_brands_reviews.attr_counts.map(lambda x: x[0][1])
chosen_brands_reviews.loc[:, 'bourbon'] = chosen_brands_reviews.attr_counts.map(lambda x: x[0][2])
chosen_brands_reviews.loc[:, 'coffee'] = chosen_brands_reviews.attr_counts.map(lambda x: x[0][3])
coc_df = chosen_brands_reviews[['name','dark', 'chocolate', 'bourbon', 'coffee']].groupby('name').sum()
coc_df

<ipython-input-25-a78cb2eac645>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen_brands_reviews.loc[:, 'attr_counts'] = chosen_brands_reviews.split_review.map(lambda x: word2vec(x, top_attributes))
<ipython-input-25-a78cb2eac645>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen_brands_reviews.loc[:, 'dark'] = chosen_brands_reviews.attr_counts.map(lambda x: x[0][0])
<ipython-input-25-a78cb2eac645>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

,dark,chocolate,bourbon,coffee
name,,,,
Ann,1,0,0,0
BDCS,19,16,29,4
Breakfast Stout,4,15,1,21
CitraQuench'l,0,0,0,0
Heady Topper,0,0,0,0
KBS - Maple Mackinac Fudge,17,46,21,27
Kaggen! Stormaktsporter,18,24,6,9
Pliny The Elder,0,0,0,0
The Rusty Nail,10,6,8,1


In [ ]:
# calculate the lifts between attributes and the top 10 brands
attr_lifts = coc_df.copy()

# iterate through df rows and columns
for i, row in attr_lifts.iterrows():
  for j in attr_lifts.columns:
    # get number of co-occurrences from the df_attr_coc dataframe
    coc = attr_lifts.loc[i, j]

    # get the count of each attribute j from the attribute frequency dataframe
    attr_count = top_attribute_freqs[j]
    # get the brand count from the top 10 brand count dataframe from before
    brand_count = grouped_beers[grouped_beers.index == i][('rating', 'count')][0]
    # lift formula
    attr_lifts.loc[i, j] = (5553*coc) / (attr_count * brand_count)
print('Lifts:')
attr_lifts

Lifts:


,dark,chocolate,bourbon,coffee
name,,,,
Ann,0.159955,0.000000,0.000000,0.000000
BDCS,2.785884,2.400778,5.275059,0.761102
Breakfast Stout,0.654700,2.512442,0.203050,4.460412
CitraQuench'l,0.000000,0.000000,0.000000,0.000000
Heady Topper,0.000000,0.000000,0.000000,0.000000
KBS - Maple Mackinac Fudge,2.062869,5.712196,3.161272,4.251673
Kaggen! Stormaktsporter,1.689125,2.304747,0.698491,1.095987
Pliny The Elder,0.000000,0.000000,0.000000,0.000000
The Rusty Nail,1.636749,1.004977,1.624397,0.212401


In [ ]:
# choosing Kaggen! Stormaktsporter, using OLS to see least different beer
def square_error(col):
  ref = col[col.index == 'Kaggen! Stormaktsporter'].values[0]
  return col.apply(lambda x: (x - ref)**2)
sq_error_df = attr_lifts.apply(square_error, axis=0)
sq_error_df.loc[:, 'sum'] = sq_error_df.sum(axis=1)
sq_error_df.sort_values(by=['sum'], inplace=True)
sq_error_df

,dark,chocolate,bourbon,coffee,sum
name,,,,,
Kaggen! Stormaktsporter,0.000000,0.000000,0.000000,0.000000,0.000000
The Rusty Nail,0.002743,1.689402,0.857302,0.780725,3.330173
Ann,2.338362,5.311859,0.487889,1.201187,9.339297
CitraQuench'l,2.853145,5.311859,0.487889,1.201187,9.854080
Heady Topper,2.853145,5.311859,0.487889,1.201187,9.854080
Pliny The Elder,2.853145,5.311859,0.487889,1.201187,9.854080
Yellow Bus,2.853145,5.311859,0.487889,1.201187,9.854080
Breakfast Stout,1.070036,0.043137,0.245462,11.319354,12.677989
BDCS,1.202879,0.009222,20.944978,0.112148,22.269227


# Most Similar Beer

For our analysis, we chose "Kaggen! Stormaktsporter" as our beer to compare others to. After analysing the difference in lifts using squared errors, we can see by our table that "The Rusty Nail" is the most similar beer.